In [46]:
import pickle
import pandas as pd
import numpy as np

## 데이터 불러오기

- cf_knn 학습할 데이터

In [47]:
# 수출 != 0 인 데이터도 포함하여 불러오기 -> CF에서는 수출이 0이면 preference도 0으로 예측
ratings_all = pd.read_csv('../processed/export_diff_label.csv')

In [48]:
ratings_all.shape

(128393, 15)

In [49]:
ratings = ratings_all.query('2011<year<2019')
ratings

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,Afghanistan,3.0,1.0,2.020357e+07,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,Afghanistan,4.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,Afghanistan,7.0,2.0,2.020357e+07,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
3,2012,Afghanistan,8.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
4,2012,Afghanistan,10.0,12.0,2.020357e+07,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80398,2018,Zimbabwe,94.0,10.0,3.415607e+07,2.927737e-07,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
80399,2018,Zimbabwe,95.0,13.0,3.415607e+07,3.806058e-07,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
80400,2018,Zimbabwe,96.0,101.0,3.415607e+07,2.957015e-06,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
80401,2018,Zimbabwe,97.0,0.0,3.415607e+07,0.000000e+00,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0


In [50]:
ratings = ratings_all.query('2011<year<2019')
ratings.drop(['수출금액'], axis=1, inplace=True)
ratings.rename(columns={'품목코드':'item_id'}, inplace=True)
ratings.rename(columns={'country':'country_id'}, inplace=True)

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38999/3486266475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings.drop(['수출금액'], axis=1, inplace=True)
/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38999/3486266475.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings.rename(columns={'품목코드':'item_id'}, inplace=True)
/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38999/3486266475.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [51]:
import pickle
with open('../processed/country_to_idx.pkl', 'rb') as f:
    country_to_idx = pickle.load(f)

with open('../processed/idx_to_country.pkl', 'rb') as f:
    idx_to_country = pickle.load(f)

In [52]:
# country_id를 고유번호로 바꾸기
ratings['country_id'] = ratings['country_id'].map(country_to_idx)

# item_id int로 바꾸기
ratings['item_id'] = ratings['item_id'].astype(int)

# 중복제거
ratings = ratings.drop_duplicates(['year', 'country_id', 'item_id'], keep='first')

# index reset
ratings.reset_index(drop=True, inplace=True)
ratings

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38999/1558890769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['country_id'] = ratings['country_id'].map(country_to_idx)
/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38999/1558890769.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['item_id'] = ratings['item_id'].astype(int)


,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,0,3,2.020357e+07,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,0,4,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,0,7,2.020357e+07,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
3,2012,0,8,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
4,2012,0,10,2.020357e+07,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80398,2018,202,94,3.415607e+07,2.927737e-07,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
80399,2018,202,95,3.415607e+07,3.806058e-07,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
80400,2018,202,96,3.415607e+07,2.957015e-06,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
80401,2018,202,97,3.415607e+07,0.000000e+00,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0


## UBCF_KNN 적용하기

In [53]:
# Rating 데이터를 test, train으로 나누고 train을 full matrix로 변환
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['country_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=12)
rating_matrix = x_train.pivot(values='preference', index=['year', 'country_id'], columns='item_id')

# Train set의 모든 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
country_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
country_similarity = pd.DataFrame(country_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# RMSE 계산을 위한 함수
def RMSE(y_true, y_pred):
    import numpy as np
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

def score(model, k=10): 
    id_pairs = zip(x_test['year'], x_test['country_id'], x_test['item_id'])
    y_pred = np.array([model(year, country, item, k) for (year, country, item) in id_pairs])
    y_true = np.array(x_test['preference'])
    return RMSE(y_true, y_pred)

In [58]:
# Neighbor size를 고려하는 추천
def cf_knn(year, country_id, item_id, neighbor_size=14): # k 디폴트 값 14(최적 K값)
    import numpy as np
    if item_id in rating_matrix:
        sim_scores = country_similarity[year, country_id]             # 연도별 현재 국가와 다른 전체 국가 간의 similarity 가져오기
        item_ratings = rating_matrix[item_id]                         # 현재 품목에 대한 모든 국가의 선호도값 가져오기
        none_rating_idx = item_ratings[item_ratings.isnull()].index   # 현재 품목에 대한 선호도가 없는 국가의 index 가져오기
        item_ratings = item_ratings.drop(none_rating_idx)             # 현재 품목에 대한 선호도가 없는 국가의 preference 값(null) 제거
        sim_scores = sim_scores.drop(none_rating_idx)                 # 현재 품목에 대한 선호도가 없는 국가의 similarity값 제거

        if neighbor_size == 0:    # Neighbor size가 지정되지 않은 경우 -> KNN 사용하지 않겠다는 의미
            # 현재 품목을 평가한 모든 국가의 가중 평균값 구하기
            mean_rating = np.dot(sim_scores, item_ratings) / (sim_scores.sum() + 0.0001)  # rating 존재하는 모든 국가 사용(simple CF사용)
        else:    # Neighbor size가 지정된 경우
            # if len(sim_scores) > 1 :
            neighbor_size = min(neighbor_size, len(sim_scores))          # 내가 지정한 K 숫자보다 품목 선호도가 있는 국가의 수가 적을 수 있음. 그럴 때, 지정된 neighbor size 값과 해당 품목 선호도가 존재하는 총국가 수 중 작은 걸 K로 결정(에러방지)
            sim_scores = np.array(sim_scores)                            # argsort()를 사용하기 위해서 array형식으로 바꿔주기
            item_ratings = np.array(item_ratings)
            country_idx = np.argsort(sim_scores)                         # 다른 국가를 유사도 순서대로 정렬(오름차순)
            sim_scores = sim_scores[country_idx][-neighbor_size:]        # 가장 유사도가 높은 k개의 국가를 선정
            item_ratings = item_ratings[country_idx][-neighbor_size:]    # 선정된 k개 국가의 주어진 품목(item_id)에 대한 평점을 추출한다
            # 최종 예측값 계산
            mean_rating = np.dot(sim_scores, item_ratings) / (sim_scores.sum() + 0.0001)  # 선정된 k 국가의 해당 품목에 대한 선호도를 유사도로 가중한 평점 평균 계산
    else:
        mean_rating = 0.000001544881   # train-set에 현재 품목에 대한 선호도가 없는 경우, 중간값을 기본값으로 지정
    return mean_rating

In [59]:
# score(cf_knn, 14)
#0.0025008920608058107

0.0025008920608058107

In [60]:
ratings_pred = pd.read_csv('../processed/nonExp_12_18.csv')
ratings_pred

,country_id,year,nonExp_items,GDP_growth,pop_growth,preference
0,Afghanistan,2012,1.0,NaN,NaN,0
1,Afghanistan,2012,2.0,NaN,NaN,0
2,Afghanistan,2012,4.0,NaN,NaN,0
3,Afghanistan,2012,5.0,NaN,NaN,0
4,Afghanistan,2012,6.0,NaN,NaN,0
...,...,...,...,...,...,...
69793,Zimbabwe,2018,89.0,0.942353,0.020411,0
69794,Zimbabwe,2018,91.0,0.942353,0.020411,0
69795,Zimbabwe,2018,92.0,0.942353,0.020411,0
69796,Zimbabwe,2018,97.0,0.942353,0.020411,0


In [61]:
ratings_pred.isnull().sum()

country_id          0
year                0
nonExp_items        0
GDP_growth      10324
pop_growth      10324
preference          0
dtype: int64

In [62]:
from tqdm import tqdm
ratings_pred = pd.read_csv('../processed/nonExp_12_18.csv')
#ratings_pred = ratings_pred[['year', 'country_id', 'nonExp_items', 'preference']]

ratings_pred_all = pd.DataFrame(columns=['country_id', 'year', 'item_id', 'GDP_growth','pop_growth','preference'])

for z in tqdm(range(2012,2019)):
    # 일부 컬럼만 가져오기
    ratings_pred_copy = ratings_pred[ratings_pred['year']==z]

    # column명 변경
    ratings_pred_copy.columns=['country_id', 'year', 'item_id','GDP_growth','pop_growth','preference']

    # country_id를 고유번호로 바꾸기
    ratings_pred_copy['country_id'] = ratings_pred_copy['country_id'].map(country_to_idx)

    # item_id int로 바꾸기
    ratings_pred_copy['item_id'] = ratings_pred_copy['item_id'].astype(int)

    # 중복제거
    ratings_pred_copy = ratings_pred_copy.drop_duplicates(['year', 'country_id', 'item_id'], keep='first')

    if z not in [2012,2013]:
        # country_id에서 133, 134 제거
        ratings_pred_copy = ratings_pred_copy[ratings_pred_copy['country_id']!=133]
        ratings_pred_copy = ratings_pred_copy[ratings_pred_copy['country_id']!=134]

    # index reset
    ratings_pred_copy.reset_index(drop=True, inplace=True)

    for i in range(len(ratings_pred_copy)):
        y = ratings_pred_copy['year'][i]
        c = ratings_pred_copy['country_id'][i]
        item = ratings_pred_copy['item_id'][i]
        ratings_pred_copy.loc[i, 'preference'] = cf_knn(y, c, item, neighbor_size=14)
    
    # 연도별 결과 합치기
    ratings_pred_all = pd.concat([ratings_pred_all, ratings_pred_copy], ignore_index=True)  

ratings_pred_all

  0%|          | 0/7 [00:00<?, ?it/s]

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38999/2274422384.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_pred_copy['country_id'] = ratings_pred_copy['country_id'].map(country_to_idx)
/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38999/2274422384.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_pred_copy['item_id'] = ratings_pred_copy['item_id'].astype(int)
 14%|█▍        | 1/7 [01:01<06:09, 61.65s/it]/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ip

,country_id,year,item_id,GDP_growth,pop_growth,preference
0,0,2012,1,NaN,NaN,8.676877e-07
1,0,2012,2,NaN,NaN,4.316267e-06
2,0,2012,4,NaN,NaN,1.464146e-06
3,0,2012,5,NaN,NaN,1.516132e-06
4,0,2012,6,NaN,NaN,1.241186e-06
...,...,...,...,...,...,...
69793,202,2018,89,0.942353,0.020411,1.069936e-05
69794,202,2018,91,0.942353,0.020411,7.333262e-07
69795,202,2018,92,0.942353,0.020411,6.773078e-07
69796,202,2018,97,0.942353,0.020411,7.280952e-10


In [63]:
ratings_pred_all.reset_index(drop=True, inplace=True)
ratings_pred_all

,country_id,year,item_id,GDP_growth,pop_growth,preference
0,0,2012,1,NaN,NaN,8.676877e-07
1,0,2012,2,NaN,NaN,4.316267e-06
2,0,2012,4,NaN,NaN,1.464146e-06
3,0,2012,5,NaN,NaN,1.516132e-06
4,0,2012,6,NaN,NaN,1.241186e-06
...,...,...,...,...,...,...
69793,202,2018,89,0.942353,0.020411,1.069936e-05
69794,202,2018,91,0.942353,0.020411,7.333262e-07
69795,202,2018,92,0.942353,0.020411,6.773078e-07
69796,202,2018,97,0.942353,0.020411,7.280952e-10


## 전체 데이터 만들기

In [64]:
nonExp = pd.read_csv('../processed/nonExp_12_18.csv')
nonExp

,country_id,year,nonExp_items,GDP_growth,pop_growth,preference
0,Afghanistan,2012,1.0,NaN,NaN,0
1,Afghanistan,2012,2.0,NaN,NaN,0
2,Afghanistan,2012,4.0,NaN,NaN,0
3,Afghanistan,2012,5.0,NaN,NaN,0
4,Afghanistan,2012,6.0,NaN,NaN,0
...,...,...,...,...,...,...
69793,Zimbabwe,2018,89.0,0.942353,0.020411,0
69794,Zimbabwe,2018,91.0,0.942353,0.020411,0
69795,Zimbabwe,2018,92.0,0.942353,0.020411,0
69796,Zimbabwe,2018,97.0,0.942353,0.020411,0


In [65]:
# nonExp country_id map함수로 idx바꾸기
nonExp['country_id'] = nonExp['country_id'].map(country_to_idx)

In [66]:
nonExp.rename(columns={'nonExp_items':'item_id'}, inplace=True)

In [67]:
nonExp.drop(['preference'], axis=1, inplace=True)

In [68]:
# all, nonExp 합치기
test12_18 = pd.merge(nonExp, ratings_pred_all, on=['country_id', 'year','item_id','GDP_growth','pop_growth'], how='left')
test12_18

,country_id,year,item_id,GDP_growth,pop_growth,preference
0,0,2012,1.0,NaN,NaN,8.676877e-07
1,0,2012,2.0,NaN,NaN,4.316267e-06
2,0,2012,4.0,NaN,NaN,1.464146e-06
3,0,2012,5.0,NaN,NaN,1.516132e-06
4,0,2012,6.0,NaN,NaN,1.241186e-06
...,...,...,...,...,...,...
69793,202,2018,89.0,0.942353,0.020411,1.069936e-05
69794,202,2018,91.0,0.942353,0.020411,7.333262e-07
69795,202,2018,92.0,0.942353,0.020411,6.773078e-07
69796,202,2018,97.0,0.942353,0.020411,7.280952e-10


In [69]:
test12_18.to_csv('../processed/test12_18_new.csv',index=False)
